# mounting


## 7th layers saved weights1 model1

In [ ]:
COLAB = False

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    directry = '/content/drive/My Drive/IML/task4/task4_handout/'
else:
    directry = 'task4_handout/'

In [ ]:
from pathlib import Path
from PIL import Image
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import copy

# Load Dataset and prepare it for training





In [ ]:
# DataFrames that have image file's names for training and validation
df_train = pd.read_csv(directry+'train_triplets.txt', sep=' ', names=('anchor', 'positive', 'negative'))
df_test = pd.read_csv(directry+'test_triplets.txt', sep=' ', names=('A', 'B', 'C')) 

# the number of output dimention in the dataset
num_dim = 108

# reshaped image size
input_size = 299

# batch size
batch_size = 32

# the number of epochs to train for
num_epochs = 15

# whther train all the layers(False) or only reshaped layers(True)
feature_extract = True

# .. 5 -> 6 -> AuxLogits -> 7 -> output
mixed = 7

In [ ]:
df_train, df_val = train_test_split(df_train, test_size=0.1)

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, dir_path, input_size, phase, df):
        super().__init__()
        self.dir_path = dir_path
        self.input_size = input_size
        self.anchor_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['anchor'])]
        self.positive_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['positive'])]
        self.negative_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['negative'])]
        self.len = len(self.anchor_image_paths) # the same as positive_image_paths and negative_image_paths

        if phase == "train":
            self.transformer = transforms.Compose([
                transforms.RandomResizedCrop(input_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        elif phase == "val":
            self.transformer = transforms.Compose([
                transforms.Resize(input_size),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        anchor = self.anchor_image_paths[index]
        positive = self.positive_image_paths[index]
        negative = self.negative_image_paths[index]
        
        # 入力
        anchor_image = Image.open(anchor)
        anchor_image = self.transformer(anchor_image)
        positive_image = Image.open(positive)
        positive_image = self.transformer(positive_image)
        negative_image = Image.open(negative)
        negative_image = self.transformer(negative_image)

            
        return anchor_image, positive_image, negative_image

In [ ]:
train_dataset = MyDataset(directry+'food/', (input_size, input_size), 'train', df_train)
val_dataset = MyDataset(directry+'food/', (input_size, input_size), 'val', df_val)

In [ ]:
train_dataloader_dict = {'train': data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=2, drop_last=True
)}

val_dataloader_dict = {'val': data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False,
    num_workers=2, drop_last=True
)}

dataloaders_dict = dict()
dataloaders_dict.update(train_dataloader_dict)
dataloaders_dict.update(val_dataloader_dict)

# Load pretrained inception model






In [ ]:
def set_parameter_requires_grad(model, feature_extracting, mixed):
    """
    """
    if feature_extracting:
        if mixed == 6:
            for name, param in model.named_parameters():
                if not(name.startswith('Mixed_6') or name.startswith('AuxLogits') or name.startswith('Mixed_7')):
                    param.requires_grad = False
        elif mixed == 7:
            for name, param in model.named_parameters():
                if not(name.startswith('Mixed_7')):
                    param.requires_grad = False

In [ ]:
def initialize_model(num_dim, feature_extract,  mixed, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model, feature_extract, mixed)
    # Handle the auxilary net
    num_ftrs = model.AuxLogits.fc.in_features
    model.AuxLogits.fc = nn.Linear(in_features=num_ftrs, out_features=num_dim)
    # Handle the primary net
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(in_features=num_ftrs, out_features=num_dim)

    return model

# Initialize the model for this run
model = initialize_model(num_dim, feature_extract, mixed, use_pretrained=True)

# Print the model we just instantiated
#print(model)

In [ ]:
PATH = directry+'initial_model1.pt'
torch.save(model, PATH)

In [ ]:
PATH = directry+'initial_model1.pt'
model = torch.load(PATH)

In [ ]:
def softmax(outputs):
  outputs = F.softmax(outputs, dim=1)
  return outputs

In [ ]:

params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer = optim.SGD(params_to_update, lr=0.008, momentum=0.9)

In [ ]:
class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin = 1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        dist_positive = (anchor - positive).pow(2).sum(1)  
        dist_negative = (anchor - negative).pow(2).sum(1)  
        losses = F.relu(dist_positive - dist_negative + self.margin)

        return losses.mean()

criterion = TripletLoss()

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs, mixed, feature_extract=True):
    since = time.time()
    
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            num_batch = 0
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for anchor, positive, negative in dataloaders[phase]:
                num_batch+=1 
                anchor = anchor.to(device)
                positive = positive.to(device)
                negative = negative.to(device)
          

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        anchor_outputs, anchor_aux_outputs = model(anchor)
                        positive_outputs, positive_aux_outputs = model(positive)
                        negative_outputs, negative_aux_outputs = model(negative)

                        anchor_outputs = softmax(anchor_outputs)
                        positive_outputs = softmax(positive_outputs)
                        negative_outputs = softmax(negative_outputs)
                        if mixed == 6:
                            anchor_aux_outputs = softmax(anchor_aux_outputs)
                            positive_aux_outputs = softmax(positive_aux_outputs)
                            negative_aux_outputs = softmax(negative_aux_outputs)
                      
                            loss1 = criterion(anchor_outputs, positive_outputs, negative_outputs)
                            loss2 = criterion(anchor_aux_outputs, positive_aux_outputs, negative_aux_outputs)

                            loss = loss1 + 0.3 * loss2

                        elif mixed == 7:
                            loss = criterion(anchor_outputs, positive_outputs, negative_outputs)
        
                    else:
                        anchor_outputs = model(anchor)
                        positive_outputs = model(positive)
                        negative_outputs = model(negative)

                        anchor_outputs = softmax(anchor_outputs)
                        positive_outputs = softmax(positive_outputs)
                        negative_outputs = softmax(negative_outputs)
                      
                        loss = criterion(anchor_outputs, positive_outputs, negative_outputs)
  
                  
                    dist_positive = (anchor_outputs - positive_outputs).pow(2).sum(1)   # should be smaller
                    dist_negative = (anchor_outputs - negative_outputs).pow(2).sum(1) # should be larger
                    diff = dist_negative - dist_positive 
                    preds = diff > 0
                  
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * anchor.size(0) # anchor.size(0) is the same as batch size
                running_corrects += torch.sum(preds == True)
                #print(f'running_loss: {running_loss / (num_batch*32)}, num_example: {num_batch*32}')
                #print()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
PATH = directry+'model3.pt'

In [ ]:
model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs, mixed)
torch.save(model_ft, PATH)

In [ ]:
model_ft = torch.load(PATH)
model_ft.eval()

# Make predictions


In [ ]:
class MyTestDataset(data.Dataset):
    def __init__(self, dir_path, input_size, df):
        super().__init__()
        self.dir_path = dir_path
        self.input_size = input_size
        self.a_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['A'])]
        self.b_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['B'])]
        self.c_image_paths = [p for p in map(lambda x: self.dir_path + str(x).zfill(5) + '.jpg', df['C'])]
        self.len = len(self.a_image_paths) # the same as positive_image_paths and negative_image_paths
        self.transformer = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        a = self.a_image_paths[index]
        b = self.b_image_paths[index]
        c = self.c_image_paths[index]
        
        # 入力
        a_image = Image.open(a)
        a_image = self.transformer(a_image)
        b_image = Image.open(b)
        b_image = self.transformer(b_image)
        c_image = Image.open(c)
        c_image = self.transformer(c_image)

        return a_image, b_image, c_image

In [ ]:
test_dataset = MyTestDataset(directry+'food/', (input_size, input_size), df_test)

In [ ]:
def predict_func(model, image1, image2, image3):
    """For one set of three images
    """
    image1_outputs = model(image1) # (1, 108)
    image2_outputs = model(image2)
    image3_outputs = model(image3)
    dist_12 = (image1_outputs - image2_outputs).pow(2).sum(1).cpu()
    dist_23 = (image1_outputs - image3_outputs).pow(2).sum(1).cpu()


    return np.where(dist_12 < dist_23, int(1),  int(0))

In [ ]:
test_dataloader = data.DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=2, drop_last=False
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
results = []

model_ft.eval()

for image1, image2, image3 in test_dataloader:
  image1 = image1.to(device)
  image2 = image2.to(device)
  image3 = image3.to(device)
  result = predict_func(model_ft, image1, image2, image3)
  results.append(result)


In [ ]:
df_predict = pd.DataFrame(results)
df_predict.to_csv(directry+'prediction1.txt', index=False, header=None, sep='\n')

In [ ]:
import pandas as pd

df = pd.read_csv(directry+ 'prediction1.txt')
df